- Option 1: $1^t + 2^t$
  - Values: $2, 3, 5, 9$
  - Characteristic polymonial is: $(3x-5)^2 - (2x-3)(5x-9)$,
    [roots from wolframalpha](https://www.wolframalpha.com/input?i=Polynomials&assumption=%7B%22C%22%2C+%22Polynomials%22%7D+-%3E+%7B%22Calculator%22%7D&assumption=%7B%22F%22%2C+%22PolynomialCalculator%22%2C+%22polynomial%22%7D+-%3E%22%283x-5%29%5E2+-+%282x-3%29%285x-9%29%22)


- Option 2: $1^t + 2^t + 3^t$
  - Values: $3, 6, 14, 36, 98, 276$
  - Characteristic polymonial is
  [wolframalpha](https://www.wolframalpha.com/input?i=Polynomials&assumption=%7B%22C%22%2C+%22Polynomials%22%7D+-%3E+%7B%22Calculator%22%7D&assumption=%7B%22F%22%2C+%22PolynomialCalculator%22%2C+%22polynomial%22%7D+-%3E%22%28-20x%5E2%2B84x-76%29%5E2-%28-6x%5E2%2B24x-20%29*%28-76x%5E2%2B336x-332%29%22):
   $$ (-20x^2 + 84x - 76)^2-(-6x^2 + 24x - 20)*(-76x^2 + 336x - 332) = -56 x^4 + 480 x^3 - 1480 x^2 + 1920 x - 864 $$

  - divided by $14x - 36$, result [wolframalpha](https://www.wolframalpha.com/input?i=Polynomials&assumption=%7B%22C%22%2C+%22Polynomials%22%7D+-%3E+%7B%22Calculator%22%7D&assumption=%7B%22F%22%2C+%22PolynomialCalculator%22%2C+%22polynomial%22%7D+-%3E%22%28-56x%5E4+%2B480x%5E3+-1480x%5E2+%2B1920x-864%29%2F%2814x-36%29%22)

    $$\frac{-56x^4 + 480x^3 - 1480x^2 + 1920x -864}{14x - 36} = -4x^3 + 24x^2 - 44x + 24$$



In [ ]:
import numpy as np
import IPython.display
#import ipywidgets
P = np.polynomial.Polynomial
Pdiv = np.polynomial.polynomial.polydiv

# Option 1: exp-polynomial: 1^t + 2^t
t = np.arange(2*2)
vals = ((1, 2)**t[:,np.newaxis]).sum(-1)

# Option 2: exp-polynomial: 1^t + 2^t + 3^t:
#t = np.arange(3*2)
#vals = ((1, 2, 3)**t[:,np.newaxis]).sum(-1)

# Option 3: regular-polynomial: t+t^2
#t = np.arange(3*2)
#vals = (t[:,np.newaxis]**(1,2,)).sum(-1)

# Option 4: exp-polynomial: t + 2^t
#t = np.arange(3*2)
#vals = (3*t[:,np.newaxis]**(1,) + (2,)**t[:,np.newaxis]).sum(-1)

vals

In [ ]:
# Display array of polynomials together with the roots
class disp_polys:
    def __init__(self, polys):
        self.polys = np.asarray(polys)
    @staticmethod
    def _get_roots(p):
        return (np.real_if_close(r) for r in p.roots())
    @staticmethod
    def _get_factors(p):
        return (f'(x {-r:+.3f})' for r in disp_polys._get_roots(p))
    def _repr_latex_(self):
        mtx = []
        for p in self.polys.flat:
            row = p._repr_latex_()[1:-1]
            row += '&' + '&'.join(self._get_factors(p))
            mtx.append(row)
        return '$\\begin{matrix}' + ' \\\\ '.join(mtx) + '\\end{matrix}$'
    def _repr_html_(self):
        mtx = []
        for p in self.polys.flat:
            row = f'<td>{p}</td><td>' + '</td><td>'.join(self._get_factors(p)) + '</td>'
            mtx.append(row)
        return '<table><tr>' + '</tr><tr>'.join(mtx) + '</tr></table>'


In [ ]:
# Convert to row of one-degree polynomials: (x*v0 - v1)
vals_p = vals[np.arange(vals.size - 1)[:, np.newaxis] + np.arange(2)[::-1]]
vals_p[...,0] *= -1
nw1 = np.apply_along_axis(P, -1, vals_p)
del vals_p
display(f'row_1:')
display(disp_polys(nw1))#, include=['text/html', 'text/latex'])


In [ ]:
nw = np.full((1 + (nw1.shape[0] + 1)//2,) + nw1.shape, None)
nw[0] = P(1)
nw[1] = nw1
def polydiv(p0, p1):
    display('polydiv:', disp_polys((p0,p1)))###
    res = np.polynomial.polynomial.polydiv(p0.coef, p1.coef)
    np.testing.assert_allclose(res[1], 0, atol=1e-10, err_msg='Non-zero polynomial division remainder')
    return P(res[0])
polydiv = np.vectorize(polydiv, otypes=nw.dtype.char)
for i in range(1, nw.shape[0]-1):
    end = nw.shape[-1]-i
    nw_sl = nw[i+1, i:end]
    display(f'row_{i+1}: {nw_sl.shape}')
    nw_sl[...] = nw[i, i:end]*nw[i, i:end] - nw[i, i-1:end-1]*nw[i, i+1:end+1]
    nw_sl[...] = polydiv(nw[i+1, i:end], nw[i-1, i:end])
    #print(f'{i:03}:', '\n     '.join(disp(nw[i+1])))
    display(disp_polys(nw_sl))
del i

In [ ]:
final_idx = -1, nw.shape[-1]//2
disp_polys(nw[final_idx])